Import Statements

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Displays all the results, not just the last one

Dataframe Conversion

In [8]:
import pandas as pd
from pathlib import Path
import pyarrow.parquet as pq

# Set month and year for processing
month = 1
year = 2024
path = Path("..") / "data" / "raw" / f"rides_{year}_{month:02}.parquet"

# Load the raw data
table = pq.read_table(path)
rides = table.to_pandas()
rides.head()
rides.shape

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5078F3D302000BD2,electric_bike,2024-01-22 18:43:19.012,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,7954.12,St Nicholas Ave & W 126 St,7756.10,40.823072,-73.941738,40.811432,-73.951878,member
1,814337105D37302A,electric_bike,2024-01-11 19:19:18.721,2024-01-11 19:47:36.007,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761822,-73.977036,40.768974,-73.954823,member
2,A33A920E2B10710C,electric_bike,2024-01-30 19:17:41.693,2024-01-30 19:32:49.857,E 11 St & Ave B,5659.11,W 10 St & Washington St,5847.06,40.727592,-73.979751,40.733424,-74.008515,casual
3,A3A5FC0DD7D34D74,electric_bike,2024-01-27 11:27:01.759,2024-01-27 11:38:01.213,W 54 St & 6 Ave,6771.13,E 74 St & 1 Ave,6953.08,40.761779,-73.977144,40.768974,-73.954823,member
4,6F96728ECEFBDAA4,electric_bike,2024-01-16 15:15:41.000,2024-01-16 15:29:26.156,Madison Ave & E 99 St,7443.01,E 74 St & 1 Ave,6953.08,40.789808,-73.952214,40.768974,-73.954823,member


(1888085, 13)

Duration and Total Amount Filter

In [9]:
# Create a mapping of station names to IDs and coordinates
# Combine start and end station information into a single mapping
start_stations = rides[['start_station_name', 'start_station_id', 'start_lat', 'start_lng']].drop_duplicates()
end_stations = rides[['end_station_name', 'end_station_id', 'end_lat', 'end_lng']].drop_duplicates()

# Rename columns to be consistent
start_stations.columns = ['station_name', 'station_id', 'latitude', 'longitude']
end_stations.columns = ['station_name', 'station_id', 'latitude', 'longitude']

# Concatenate and remove duplicates
station_mapping = pd.concat([start_stations, end_stations], ignore_index=True).drop_duplicates(subset=['station_name', 'station_id'])

# Save the mapping to a CSV file in the data folder
mapping_path = Path("..") / "data" / "mapping.csv"
station_mapping.to_csv(mapping_path, index=False)
print(f"Station mapping saved to: {mapping_path}")
station_mapping.head()

# Drop unnecessary columns from the main DataFrame
rides = rides.drop(columns=[
    'ride_id',  # Remove ride_id as requested
    'start_station_id', 'end_station_id',  # Remove station IDs
    'start_lat', 'start_lng', 'end_lat', 'end_lng'  # Remove coordinates
])

# Reset index to use an integer-based index
rides = rides.reset_index(drop=True)

# Compute duration
rides['duration'] = rides['ended_at'] - rides['started_at']

# Apply validation filters
# Duration filter: Between 1 minute and 2 hours
duration_filter = (rides['duration'] >= pd.Timedelta(minutes=1)) & (rides['duration'] <= pd.Timedelta(hours=2))
invalid_duration = sum(~duration_filter) / rides.shape[0] * 100
print(f"Invalid duration rows: {invalid_duration:.2f}%")

# Station filter: Ensure start_station_name is not null
station_filter = rides['start_station_name'].notnull()
invalid_stations = sum(~station_filter) / rides.shape[0] * 100
print(f"Invalid station rows: {invalid_stations:.2f}%")

# Date range filter: Ensure started_at is within January 2024
filter_date_range = (rides['started_at'] >= f"{year}-{month:02}-01") & (rides['started_at'] < f"{year}-{month:02}-01".replace('01', '02'))
invalid_date_range = sum(~filter_date_range) / rides.shape[0] * 100
print(f"Invalid date range rows: {invalid_date_range:.2f}%")

Station mapping saved to: ..\data\mapping.csv


,station_name,station_id,latitude,longitude
0,Frederick Douglass Blvd & W 145 St,7954.12,40.823072,-73.941738
1,W 54 St & 6 Ave,6771.13,40.761822,-73.977036
2,E 11 St & Ave B,5659.11,40.727592,-73.979751
4,Madison Ave & E 99 St,7443.01,40.789808,-73.952214
6,Butler St & Court St,4339.01,40.685106,-73.994274


Invalid duration rows: 0.18%
Invalid station rows: 0.06%
Invalid date range rows: 0.02%


In [10]:
# Combine all filters
final_filter = duration_filter & station_filter & filter_date_range
numbers_dropped = final_filter.shape[0] - sum(final_filter)
rows_dropped = numbers_dropped / final_filter.shape[0] * 100
print(f"Total invalid rows: {rows_dropped:.2f}%")
print(f"Number of rows dropped: {numbers_dropped}")

Total invalid rows: 0.26%
Number of rows dropped: 4926


In [11]:
# Apply the filter and reset index
rides = rides[final_filter].reset_index(drop=True)

# Inspect the filtered DataFrame
rides.head()
rides.shape

,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,duration
0,electric_bike,2024-01-22 18:43:19.012,2024-01-22 18:48:10.708,Frederick Douglass Blvd & W 145 St,St Nicholas Ave & W 126 St,member,0 days 00:04:51.696000
1,electric_bike,2024-01-11 19:19:18.721,2024-01-11 19:47:36.007,W 54 St & 6 Ave,E 74 St & 1 Ave,member,0 days 00:28:17.286000
2,electric_bike,2024-01-30 19:17:41.693,2024-01-30 19:32:49.857,E 11 St & Ave B,W 10 St & Washington St,casual,0 days 00:15:08.164000
3,electric_bike,2024-01-27 11:27:01.759,2024-01-27 11:38:01.213,W 54 St & 6 Ave,E 74 St & 1 Ave,member,0 days 00:10:59.454000
4,electric_bike,2024-01-16 15:15:41.000,2024-01-16 15:29:26.156,Madison Ave & E 99 St,E 74 St & 1 Ave,member,0 days 00:13:45.156000


(1883159, 7)

In [12]:
# Save the filtered data to the processed folder
processed_path = Path("..") / "data" / "processed" / f"rides_{year}_{month:02}.parquet"
processed_path.parent.mkdir(parents=True, exist_ok=True)
rides.to_parquet(processed_path, engine="pyarrow", index=False)
print(f"Filtered data saved to: {processed_path}")

Filtered data saved to: ..\data\processed\rides_2024_01.parquet
